In [ ]:
# Set environment variables
export INSTANCE_ID="wordpress-db-instance"
export REGION="us-east1"
export ZONE="us-east1-d"
export ROOT_PASSWORD="admin123"
export BUCKET_NAME="qwiklabs-gcp-04-c7628d0dc4de"

##Task 1. Create a new Cloud SQL instance

In [ ]:
# Create Cloud SQL instance
gcloud sql instances create $INSTANCE_ID \
    --database-version=MYSQL_8_0 \
    --tier=db-f1-micro \
    --region=$REGION \
    --root-password=$ROOT_PASSWORD \
    --storage-size=10GB

##Task 2. Configure the new database

In [ ]:
# Set variables
export DB_NAME="wordpress"
export DB_USER="blogadmin"
export DB_PASSWORD="Password1*"

In [ ]:
# Create SQL DB
gcloud sql databases create $DB_NAME --instance=$INSTANCE_ID

In [ ]:
# Create DB user
gcloud sql users create $DB_USER \
    --instance=$INSTANCE_ID \
    --password=$DB_PASSWORD

In [ ]:
# Verify configuration
gcloud sql databases list --instance=$INSTANCE_ID
gcloud sql users list --instance=$INSTANCE_ID

##Task 3. Perform a database dump and import the data

In [ ]:
# Export the current DB
mysqldump -u blogadmin -p wordpress > wordpress_db.sql

In [ ]:
# Upload the SQL dump to a Cloud Stotage bucket
gcloud storage buckets create gs://${BUCKET_NAME} --location=REGION

In [ ]:
gsutil cp wordpress_db.sql gs://$BUCKET_NAME/

In [ ]:
# Import the SQL dump into Cloud SQL
gcloud sql import sql $INSTANCE_ID gs://$BUCKET_NAME/wordpress_db.sql \
    --database=$DB_NAME

In [ ]:
# Connect to DB, check if the tables are imported
gcloud sql connect $INSTANCE_ID --user=$DB_USER

##Task 4. Reconfigure the WordPress installation

In [ ]:
# Update wp-config.php, locate and edit the wp-config.php file on your WordPress server
vi /var/www/html/wp-config.php

In [ ]:
# Update the DB with the following details
define('DB_NAME', 'wordpress');
define('DB_USER', 'blogadmin');
define('DB_PASSWORD', 'Password1*');
define('DB_HOST', '34.148.119.26'); # Cloud SQL -> Public IP address

In [ ]:
# Restart the webserver
sudo systemctl restart apache2

##Task 5. Validate and troubleshoot

In [ ]:
# Test DB connectivity
gcloud sql connect $INSTANCE_ID --user=$DB_USER

In [ ]:
# Check logs for errors in WordPress
tail -f /var/log/apache2/error.log

In [ ]:
# Ensure that the WordPress server is authorized to access the DB
gcloud sql instances patch $INSTANCE_ID \
    --authorized-networks=<WORDPRESS_SERVER_PUBLIC_IP>